# Import Libraries

In [1]:
import os
import sys
import pandas as pd

from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\nperc\Documents\Datathon\01_iHeath_env2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Prepare data

In [2]:
path_dir = os.path.dirname(os.path.abspath(''))

In [3]:
sys.path.append(os.path.join(path_dir, 'scripts\\common'))
import tool

In [9]:
binary_columns      = ['Sex']

data = pd.read_excel('C:\\Users\\nperc\\Documents\\Datathon\\data\\Tableur données datathon.xlsx')

# y = data['CHA2DS2-VASc']
# X = data.drop('CHA2DS2-VASc', axis=1)

In [10]:
data.columns

Index(['Num patient', 'Num etude', 'DDN', 'date d'inclusion', 'Age ', 'Female',
       'Congestive_Heart_Failure', 'HTA', 'DIABETES', 'Stroke',
       'vascular_disease', 'poids', 'taille', 'IMC', 'CHA2DS2_VASc', 'SAS',
       'Tabac ', 'DYSLIPIDEMIE', 'Longueur_axiale', 'Unnamed: 19', 'PIO',
       'Unnamed: 21', 'Volume_OCTA', 'Angio_en_face_OCTA_sup',
       'Angio_en_face_OCTA_deep', 'Angio_en_face_OCTA_cc', 'SIDE_OCTA',
       'FAZRawLength_OD', 'FazCircularity_OD', 'FAZRawSize_OD',
       'FAZRawLength_OG', 'FazCircularity_OG', 'FAZRawSize_OG',
       'DENS_Average_FOV_sup_OD', 'DENS_Circle3mm_sup_OD',
       'DENS_Circle6mm_sup_OD', 'DENS_Average_FOV_sup_OG',
       'DENS_Circle3mm_sup_OG', 'DENS_Circle6mm_sup_OG',
       'PERF_Average_FOV_sup_OD', 'PERF_Circle3mm_sup_OD',
       'PERF_Circle6mm_sup_OD', 'PERF_Average_FOV_sup_OG',
       'PERF_Circle3mm_sup_OG', 'PERF_Circle6mm_sup_OG',
       'DENS_Average_FOV_deep_OD', 'DENS_Circle3mm_deep_OD',
       'DENS_Circle6mm_deep_OD',

## Preprocessing

In [5]:
preprocessor = tool.make_preprocess(X, binary_columns)

X = preprocessor.fit_transform(X)

## Model definition

In [6]:
lr_model = LogisticRegression

lr_param = {"C":[0.001, 0.01, 0.1, 1, 10, 100, 1000], "solver":["newton-cg", "lbfgs", "liblinear"], "penalty":["l2"]}

tree_model = DecisionTreeClassifier

tree_param = {
    'max_features'  : ['sqrt', 'log2'],
    'ccp_alpha'     : [0.1, .01, .001],
    'max_depth'     : [2, 4, 6, 8, 10, 12],
    'criterion'     : ['gini', 'entropy']
    }

forest_model = RandomForestClassifier

forest_param = {
    'bootstrap'         : [True, False],
    'max_depth'         : [8, 10, 12, 15],
    'max_features'      : ['sqrt', 'log2'],
    'ccp_alpha'         : [0.1, .01, .001],
    'n_estimators'      : [20, 50],
    'criterion'         : ['gini', 'entropy']
    }

svc_model = SVC

svc_param = {
    'C'         :[0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma'     :[1,0.1,0.001,0.0001],
    'kernel'    :['linear', 'poly', 'rbf', 'sigmoid']
}

### Split dataset to find best parameters

In [10]:
nb_fold = 10

In [11]:
index_train, index_test             = tool.dataset_split_index(X, y, fold=nb_fold)

## Find best parameters

In [12]:
k_neighbors_smote = None

In [13]:
lr_report, lr_index_fold                = tool.find_best_grid(lr_model(random_state=42), X, y, index_train, index_test, param=lr_param, k_neighbors_smote=k_neighbors_smote)
tree_report, tree_index_fold            = tool.find_best_grid(tree_model(random_state=42), X, y, index_train, index_test, param=tree_param, k_neighbors_smote=k_neighbors_smote)
forest_report, forest_index_fold        = tool.find_best_grid(forest_model(random_state=42), X, y, index_train, index_test, param=forest_param, k_neighbors_smote=k_neighbors_smote)
svc_report, svc_index_fold              = tool.find_best_grid(svc_model(random_state=42), X, y, index_train, index_test, param=svc_param, k_neighbors_smote=k_neighbors_smote)

### Function to train model and display the metrics

In [ ]:
def train_pred(model, report, X, y, index_fold) :
    
    X_train, y_train, X_test, y_test = tool.split_train(X, y, index_train[index_fold], index_test[index_fold])
    
    model       = model(**report['best_grid'], random_state=42)
    
    model.fit(X_train, y_train)
    y_pred      = model.predict(X_test)
    
    print('Mean accuracy : ' + str(report['best_mean_score']))
    print('Standard deviation : ' + str(report['best_st_score']))
    print('Accuracy : ' + str(accuracy_score(y_test, y_pred)))
    print(confusion_matrix(y_test, y_pred))

## Logistic Regression

In [ ]:
train_pred(lr_model, lr_report, X, y, lr_index_fold)

Mean accuracy : 0.7353877551020408
Standard deviation : 0.03627515284418828
Accuracy : 0.7959183673469388
[[26  3  0]
 [ 4  9  0]
 [ 0  3  4]]


## Decision Tree

In [ ]:
train_pred(tree_model, tree_report, X, y, tree_index_fold)

Mean accuracy : 0.6707346938775511
Standard deviation : 0.02662091382531086
Accuracy : 0.7142857142857143
[[25  1  0]
 [ 5  9  1]
 [ 4  3  1]]


## Random Forest

In [ ]:
train_pred(forest_model, forest_report, X, y, forest_index_fold)

Mean accuracy : 0.6990612244897959
Standard deviation : 0.03729238795094402
Accuracy : 0.7959183673469388
[[27  2  0]
 [ 4  9  0]
 [ 1  3  3]]


## SVC

In [ ]:
train_pred(svc_model, svc_report, X, y, svc_index_fold)

Mean accuracy : 0.7313469387755103
Standard deviation : 0.05324733452392279
Accuracy : 0.82
[[28  4  0]
 [ 2 13  0]
 [ 0  3  0]]
